In [1]:
!python -m pip install --upgrade pip
!pip install --user keras
!pip install --user numpy
!pip install --user tensorflow

Requirement already up-to-date: pip in c:\users\mayingzh\appdata\roaming\python\python36\site-packages (19.2.2)


In [2]:
import pandas as pd
import numpy as np
from keras.models import load_model, Sequential
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K
import sklearn.metrics as skm, math
#from sklearn.utils.validation import check_array

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mayingzh\AppData\R

In [3]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))
def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))
def mpe(y_true, y_pred):
    errors = K.sqrt(K.mean((y_true-y_pred)**2))/K.mean(y_true)
    return errors

def exp(x):
    return K.exp(x)

### With Implied Volatility

In [4]:
#Load saved model
model = load_model('model4_0.h5', custom_objects={'r_square': r_square})
model.summary()

W0820 19:41:55.861617 13880 deprecation_wrapper.py:119] From C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 19:41:55.893690 13880 deprecation_wrapper.py:119] From C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 19:41:55.946478 13880 deprecation_wrapper.py:119] From C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0820 19:41:55.947487 13880 deprecation_wrapper.py:119] From C:\Users\mayingzh\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:184: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.Confi

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 30)                180       
_________________________________________________________________
dense_3 (Dense)              (None, 60)                1860      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 61        
Total params: 2,146
Trainable params: 2,146
Non-trainable params: 0
_________________________________________________________________


In [5]:
def intrinsic(S, K, C, P):
    return C*(max(S-K, 0)) + P*(max(K-S, 0))

In [22]:
def load_data():
    df = pd.read_csv('Options_R_test.csv')
    df = df.dropna()
    
    # Calculate the intrinsic value of all the options
    df['intrinsic'] = df.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], row['cp_flag_P']), axis = 1)
    
    df[['strike_price', 'underlying_price', 'maturity', 'impl_volatility', 'realized_vol', 'cp_flag_C', 'cp_flag_P', 
                    'interest_rate', 'intrinsic', 'best_offer']].apply(pd.to_numeric)
    #sample_df = sample_df.reset_index(drop=True)
    
    return df

In [23]:
df = load_data()
df.head()

,ticker,date,exdate,maturity,strike_price,best_offer,impl_volatility,realized_vol,underlying_price,interest_rate,cp_flag_C,cp_flag_P,intrinsic
0,AXP,2016-01-04,2016-01-08,4,100.0,0.13,1.695791,0.208476,67.59,0.0022,1,0,0.00
1,AXP,2016-01-04,2016-01-08,4,105.0,0.13,1.869624,0.208476,67.59,0.0022,1,0,0.00
2,AXP,2016-01-04,2016-01-08,4,66.0,1.70,0.238987,0.208476,67.59,0.0022,1,0,1.59
3,AXP,2016-01-04,2016-01-08,4,66.5,1.31,0.246213,0.208476,67.59,0.0022,1,0,1.09
4,AXP,2016-01-04,2016-01-08,4,67.0,0.95,0.242758,0.208476,67.59,0.0022,1,0,0.59


In [16]:
def mean_sum_error(y_true, y_pred):
    errors = np.sum(np.abs(y_true-y_pred))/np.sum(y_true)
    return errors

In [19]:
def model_pred(df, model, vol):
    
    y = df['best_offer'].values
    X = df[['strike_price', 'underlying_price', 'maturity', vol, 'cp_flag_C', 'cp_flag_P', 'interest_rate', 'intrinsic']]
    
    sc_X = StandardScaler()
    X_scale = sc_X.fit_transform(X)

    sc_y = StandardScaler()
    y = y.reshape(-1,1)
    y_scale = sc_y.fit_transform(y)
    
    pred = model.predict(X_scale)
    print('y_scale data:\n', y_scale[:10])
    print('scaled y_predition:\n', pred[:10])
    print('\n')
    
    y_pred = sc_y.inverse_transform(pred)
    print('actual value:\n', y[:10])
    print('actual predicted value:\n', y_pred[:10])
    print('\n')
    
    print("Average sum error:              %f" % mean_sum_error(y, y_pred))
    print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y, y_pred))
    print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y, y_pred))
    print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y, y_pred)))
    print("R square (R^2):                 %f" % skm.r2_score(y, y_pred)) 

In [24]:
model_pred(df, model, 'impl_volatility')

y_scale data:
 [[-0.50128955]
 [-0.50128955]
 [-0.39568243]
 [-0.42191605]
 [-0.44613169]
 [-0.46563874]
 [-0.48043719]
 [-0.4911997 ]
 [-0.49859892]
 [-0.50330752]]
scaled y_predition:
 [[-0.5618566 ]
 [-0.5618566 ]
 [-0.39006302]
 [-0.4191656 ]
 [-0.45024619]
 [-0.48489285]
 [-0.49846062]
 [-0.48393595]
 [-0.48339933]
 [-0.48714858]]


actual value:
 [[0.13]
 [0.13]
 [1.7 ]
 [1.31]
 [0.95]
 [0.66]
 [0.44]
 [0.28]
 [0.17]
 [0.1 ]]
actual predicted value:
 [[-0.77041626]
 [-0.77041626]
 [ 1.7835407 ]
 [ 1.3508892 ]
 [ 0.8888321 ]
 [ 0.37376025]
 [ 0.17205621]
 [ 0.38798574]
 [ 0.3959637 ]
 [ 0.34022573]]


Average sum error:              0.107699
Mean absolute error (MAE):      0.816619
Mean squared error (MSE):       1.217904
Root mean squared error (RMSE): 1.103587
R square (R^2):                 0.994489


### With Realized Volatility

In [21]:
model_pred(df, model, 'realized_vol')

y_scale data:
 [[-0.50128955]
 [-0.50128955]
 [-0.39568243]
 [-0.42191605]
 [-0.44613169]
 [-0.46563874]
 [-0.48043719]
 [-0.4911997 ]
 [-0.49859892]
 [-0.50330752]]
scaled y_predition:
 [[-0.5618566 ]
 [-0.5618566 ]
 [-0.38714704]
 [-0.41758773]
 [-0.44802856]
 [-0.47846925]
 [-0.4833054 ]
 [-0.482521  ]
 [-0.48173666]
 [-0.48431903]]


actual value:
 [[0.13]
 [0.13]
 [1.7 ]
 [1.31]
 [0.95]
 [0.66]
 [0.44]
 [0.28]
 [0.17]
 [0.1 ]]
actual predicted value:
 [[-0.77041626]
 [-0.77041626]
 [ 1.826891  ]
 [ 1.3743467 ]
 [ 0.92180014]
 [ 0.46925595]
 [ 0.39735988]
 [ 0.4090214 ]
 [ 0.4206815 ]
 [ 0.38229087]]


Average sum error:              0.217965
Mean absolute error (MAE):      1.652692
Mean squared error (MSE):       8.815138
Root mean squared error (RMSE): 2.969030
R square (R^2):                 0.960114


#### Predict special ticker which was used to train the model

In [25]:
MCD = df[df.ticker == 'MCD']
model_pred(MCD, model, 'realized_vol')

y_scale data:
 [[-0.37136029]
 [-0.44322478]
 [-0.50435089]
 [-0.55308658]
 [-0.58695375]
 [-0.60677843]
 [-0.61669078]
 [-0.62164695]
 [-0.62329901]
 [-0.61669078]]
scaled y_predition:
 [[-0.5432026]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]]


actual value:
 [[3.1 ]
 [2.23]
 [1.49]
 [0.9 ]
 [0.49]
 [0.25]
 [0.13]
 [0.07]
 [0.05]
 [0.13]]
actual predicted value:
 [[1.0196568]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]]


Average sum error:              0.563745
Mean absolute error (MAE):      4.282053
Mean squared error (MSE):       48.022099
Root mean squared error (RMSE): 6.929798
R square (R^2):                 0.672334


In [26]:
model_pred(MCD, model, 'impl_volatility')

y_scale data:
 [[-0.37136029]
 [-0.44322478]
 [-0.50435089]
 [-0.55308658]
 [-0.58695375]
 [-0.60677843]
 [-0.61669078]
 [-0.62164695]
 [-0.62329901]
 [-0.61669078]]
scaled y_predition:
 [[-0.5310955]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]
 [-0.5618566]]


actual value:
 [[3.1 ]
 [2.23]
 [1.49]
 [0.9 ]
 [0.49]
 [0.25]
 [0.13]
 [0.07]
 [0.05]
 [0.13]]
actual predicted value:
 [[1.1662265]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]
 [0.7938286]]


Average sum error:              0.548850
Mean absolute error (MAE):      4.168918
Mean squared error (MSE):       49.166730
Root mean squared error (RMSE): 7.011899
R square (R^2):                 0.664524


In [30]:
MCD.shape

(39975, 13)

In [28]:
GS = df[df.ticker == 'GS']
model_pred(GS, model, 'realized_vol')

y_scale data:
 [[-0.42571587]
 [-0.50607751]
 [-0.56706993]
 [-0.6074568 ]
 [-0.63094713]
 [-0.64166201]
 [-0.64537101]
 [-0.64743157]
 [-0.6449589 ]
 [-0.64908001]]
scaled y_predition:
 [[2.190856 ]
 [2.047441 ]
 [1.9155408]
 [1.8520836]
 [1.7886263]
 [1.7251688]
 [1.6617125]
 [1.598256 ]
 [1.5347985]
 [1.471342 ]]


actual value:
 [[5.45]
 [3.5 ]
 [2.02]
 [1.04]
 [0.47]
 [0.21]
 [0.12]
 [0.07]
 [0.13]
 [0.03]]
actual predicted value:
 [[68.941925]
 [65.461914]
 [62.26132 ]
 [60.721508]
 [59.1817  ]
 [57.641884]
 [56.102097]
 [54.562305]
 [53.02249 ]
 [51.482697]]


Average sum error:              0.471659
Mean absolute error (MAE):      7.442835
Mean squared error (MSE):       185.168572
Root mean squared error (RMSE): 13.607666
R square (R^2):                 0.685518


In [29]:
model_pred(GS, model, 'impl_volatility')

y_scale data:
 [[-0.42571587]
 [-0.50607751]
 [-0.56706993]
 [-0.6074568 ]
 [-0.63094713]
 [-0.64166201]
 [-0.64537101]
 [-0.64743157]
 [-0.6449589 ]
 [-0.64908001]]
scaled y_predition:
 [[2.2082686]
 [2.0242443]
 [1.8931569]
 [1.8330992]
 [1.7715718]
 [1.7031518]
 [1.6234287]
 [1.5501508]
 [1.4062109]
 [1.3517148]]


actual value:
 [[5.45]
 [3.5 ]
 [2.02]
 [1.04]
 [0.47]
 [0.21]
 [0.12]
 [0.07]
 [0.13]
 [0.03]]
actual predicted value:
 [[69.36445 ]
 [64.89904 ]
 [61.718166]
 [60.26085 ]
 [58.767864]
 [57.107635]
 [55.17313 ]
 [53.395016]
 [49.90227 ]
 [48.579906]]


Average sum error:              0.457963
Mean absolute error (MAE):      7.226720
Mean squared error (MSE):       173.136415
Root mean squared error (RMSE): 13.158131
R square (R^2):                 0.705953


In [31]:
NKE = df[df.ticker == 'NKE']
model_pred(NKE, model, 'realized_vol')

y_scale data:
 [[-0.05944965]
 [-0.18904377]
 [-0.27587182]
 [-0.35492423]
 [-0.4106497 ]
 [-0.45989546]
 [-0.50784529]
 [-0.54672352]
 [-0.57912205]
 [-0.60504087]]
scaled y_predition:
 [[ 0.36649692]
 [ 0.19608265]
 [ 0.12372702]
 [ 0.05137134]
 [-0.02104038]
 [-0.09413278]
 [-0.16722542]
 [-0.24031782]
 [-0.27411678]
 [-0.3062786 ]]


actual value:
 [[4.55]
 [3.55]
 [2.88]
 [2.27]
 [1.84]
 [1.46]
 [1.09]
 [0.79]
 [0.54]
 [0.34]]
actual predicted value:
 [[7.836774 ]
 [6.5217896]
 [5.9634643]
 [5.405139 ]
 [4.846381 ]
 [4.282371 ]
 [3.718359 ]
 [3.1543489]
 [2.8935425]
 [2.6453693]]


Average sum error:              0.538707
Mean absolute error (MAE):      2.698240
Mean squared error (MSE):       18.241188
Root mean squared error (RMSE): 4.270970
R square (R^2):                 0.693646


In [32]:
KO = df[df.ticker == 'KO']
model_pred(KO, model, 'realized_vol')

y_scale data:
 [[-0.31796112]
 [-0.43220254]
 [-0.51924363]
 [-0.5736443 ]
 [-0.59268454]
 [-0.5899645 ]
 [-0.58724447]
 [-0.5899645 ]
 [-0.5899645 ]
 [-0.58724447]]
scaled y_predition:
 [[-0.37044603]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]]


actual value:
 [[1.08]
 [0.66]
 [0.34]
 [0.14]
 [0.07]
 [0.08]
 [0.09]
 [0.08]
 [0.08]
 [0.09]]
actual predicted value:
 [[0.8870432 ]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]
 [0.18333644]]


Average sum error:              0.471124
Mean absolute error (MAE):      1.059539
Mean squared error (MSE):       3.131246
Root mean squared error (RMSE): 1.769533
R square (R^2):                 0.768332


In [33]:
AXP = df[df.ticker == 'AXP']
model_pred(AXP, model, 'realized_vol')

y_scale data:
 [[-0.55472657]
 [-0.55472657]
 [-0.36969014]
 [-0.41565461]
 [-0.45808334]
 [-0.49226204]
 [-0.51819071]
 [-0.53704793]
 [-0.55001227]
 [-0.5582623 ]]
scaled y_predition:
 [[-0.5156728 ]
 [-0.5618566 ]
 [ 1.3395756 ]
 [ 1.2561995 ]
 [ 1.1728238 ]
 [ 1.0894476 ]
 [ 1.0447448 ]
 [ 1.0085309 ]
 [ 0.9723166 ]
 [ 0.93143225]]


actual value:
 [[0.13]
 [0.13]
 [1.7 ]
 [1.31]
 [0.95]
 [0.66]
 [0.44]
 [0.28]
 [0.17]
 [0.1 ]]
actual predicted value:
 [[ 0.4613642 ]
 [ 0.06950276]
 [16.202805  ]
 [15.495375  ]
 [14.787947  ]
 [14.080516  ]
 [13.7012205 ]
 [13.393951  ]
 [13.08668   ]
 [12.739784  ]]


Average sum error:              0.522928
Mean absolute error (MAE):      2.529273
Mean squared error (MSE):       17.839835
Root mean squared error (RMSE): 4.223723
R square (R^2):                 0.752197


#### Predict special ticker which was not used to train the model

In [34]:
AAPL = pd.read_csv('Options2016AAPL_withReaVo.csv')
AAPL = AAPL.dropna()
AAPL.rename(columns = {'Strike Price':'strike_price', 'Lowest  Closing Ask Across All Exchanges':'best_offer', 
                       'Underlying Price':'underlying_price', 'C=Call, P=Put_C':'cp_flag_C', 'C=Call, P=Put_P':'cp_flag_P',
                       'volatility':'realized_vol'}, inplace = True);
AAPL.head()

,Unnamed: 0,strike_price,best_offer,underlying_price,maturity,cp_flag_C,cp_flag_P,interest_rate,days,realized_vol
0,0,100.0,5.45,105.35,4,1,0,0.0022,30,0.218791
1,1,101.0,4.50,105.35,4,1,0,0.0022,30,0.218791
2,2,102.0,3.65,105.35,4,1,0,0.0022,30,0.218791
3,3,103.0,2.79,105.35,4,1,0,0.0022,30,0.218791
4,4,104.0,2.06,105.35,4,1,0,0.0022,30,0.218791


In [35]:
AAPL['intrinsic'] = AAPL.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], 
                                                     row['cp_flag_P']), axis = 1)

In [36]:
model_pred(AAPL, model, 'realized_vol')

y_scale data:
 [[-0.36548695]
 [-0.40373587]
 [-0.43795859]
 [-0.47258393]
 [-0.5019752 ]
 [-0.52734027]
 [-0.54747128]
 [-0.56236823]
 [-0.57203112]
 [-0.5776678 ]]
scaled y_predition:
 [[-0.47600728]
 [-0.54169834]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]
 [-0.5618566 ]]


actual value:
 [[5.45]
 [4.5 ]
 [3.65]
 [2.79]
 [2.06]
 [1.43]
 [0.93]
 [0.56]
 [0.32]
 [0.18]]
actual predicted value:
 [[2.7049732]
 [1.0733843]
 [0.5727072]
 [0.5727072]
 [0.5727072]
 [0.5727072]
 [0.5727072]
 [0.5727072]
 [0.5727072]
 [0.5727072]]


Average sum error:              0.403199
Mean absolute error (MAE):      5.857563
Mean squared error (MSE):       102.693243
Root mean squared error (RMSE): 10.133767
R square (R^2):                 0.833531
